This notebook is to test the vanilla transformer training and validation task for language translation

In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir("/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers")
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers


In [3]:
from S16_code.config import get_config, get_weights_file_path
from S16_code.model import build_transformer
import torch
from S16_code.dataloader import get_ds
from S16_code.trainer import run_validation_loop, run_training_loop, save_model
from datasets import load_dataset
import torch.optim as optim
import torch.nn as nn

In [4]:
device = get_config()["device"]
print(device)

cuda


In [5]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name())
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())
print(torch.cuda.get_device_name(device.index))
print(torch.cuda.get_device_properties(device.index).total_memory / 1024**3)
print(torch.cuda.memory_summary(device=device.index))
print(torch.cuda.get_device_capability(device.index))
print(torch.cuda.get_device_properties(device.index))

True
0
Tesla V100-PCIE-16GB
0
0
Tesla V100-PCIE-16GB
15.7725830078125
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 

In [6]:
ds_raw = load_dataset(
    get_config()["datasource"],
    f"{get_config()['src_lang']}-{get_config()['tgt_lang']}",
    split="train",
)
print(len(ds_raw))

32332


In [7]:
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(
    ds_raw, get_config()
)

Building tokenizer for en
Building tokenizer for it
Max length of source sentence: 309
Max length of target sentence: 274


In [8]:
for data in train_dataloader:
    print(data['encoder_input'].shape)
    print(data['decoder_input'].shape)
    print(data['encoder_mask'].shape)
    print(data['decoder_mask'].shape)
    break

torch.Size([16, 350])
torch.Size([16, 350])
torch.Size([16, 1, 1, 350])
torch.Size([16, 1, 350, 350])


In [9]:
for data in val_dataloader:
    print(data['encoder_input'].shape)
    print(data['decoder_input'].shape)
    print(data['encoder_mask'].shape)
    print(data['decoder_mask'].shape)
    break

torch.Size([1, 350])
torch.Size([1, 350])
torch.Size([1, 1, 1, 350])
torch.Size([1, 1, 350, 350])


In [10]:
print(tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size())

15698 22463


In [11]:
transformer_model = build_transformer(
    src_vocab_size=tokenizer_src.get_vocab_size(),
    tgt_vocab_size=tokenizer_tgt.get_vocab_size(),
).to(device)

In [12]:
print(torch.cuda.memory_summary(device=device.index))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 295276 KiB | 295276 KiB | 295276 KiB |      0 B   |
|       from large pool | 219812 KiB | 219812 KiB | 219812 KiB |      0 B   |
|       from small pool |  75464 KiB |  75464 KiB |  75464 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         | 295276 KiB | 295276 KiB | 295276 KiB |      0 B   |
|       from large pool | 219812 KiB | 219812 KiB | 219812 KiB |

In [13]:
!nvidia-smi

Sun Jun  9 05:45:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           On  | 00000001:00:00.0 Off |                    0 |
| N/A   29C    P0              36W / 250W |    604MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
optimizer = optim.Adam(transformer_model.parameters(), lr=get_config()["learning_rate"])
loss_fn = nn.CrossEntropyLoss(
    ignore_index=tokenizer_src.token_to_id("[PAD]"), label_smoothing=0.1
)
global_step = 0

In [15]:
for epoch in range(get_config()["num_epochs"]):
    print(f"Epoch {epoch + 1}")
    torch.cuda.empty_cache()
    epoch_loss, global_step = run_training_loop(
        transformer_model,
        train_dataloader,
        tokenizer_tgt,
        optimizer,
        loss_fn,
        device,
        global_step,
    )
    print(f"Epoch loss: {epoch_loss}")
    run_validation_loop(
        transformer_model,
        val_dataloader,
        tokenizer_src,
        tokenizer_tgt,
        device,
        get_config()["seq_len"],
        2,
    )
    model_path = get_weights_file_path(get_config(), epoch)
    save_model(epoch, transformer_model, optimizer, global_step, model_path)

Epoch 1


Training:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0/1819 [00:00<?, ?it/s]

Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1819/1819 [12:11<00:00,  2.49it/s, loss=5.698]


Epoch loss: 5.697751522064209


Validation:   0%|▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 1/3234 [00:00<09:46,  5.51it/s]

...

Epoch loss: 5.959231853485107


Validation:   0%|▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 1/3234 [00:06<5:24:21,  6.02s/it]

SOURCE: He rose and went toward the door.
TARGET: Si alzò e si diresse verso la porta.
PREDICTED: — Sì , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signore , signo

Validation:   0%|▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 1/3234 [00:12<10:57:45, 12.21s/it]


SOURCE: It was the manner in which all this, and much more, was said—it was the apparent heart that went with his request—which allowed me no room for hesitation; and I accordingly obeyed forthwith what I still considered a very singular summons.
TARGET: Era il tono con cui eran dette tutte queste cose e ben altre ancora, era quell'espansione d'un cuore supplichevole, che non mi permetteva d'esitare; quindi obbedii, immediatamente a quello che consideravo tuttavia come uno degli inviti più singolari.
PREDICTED: — No , non mi , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse , — disse

# Next steps: 
* Check prediction loss for both tran and val
* add metrics from torch metrics
* add tensorboard logging
* add gradient clipping
* add learning rate scheduler
* add AMP - automatic mixed precision
* add distributed training
* add model parallelism
* add data parallelism
* add pipeline parallelism
* add checkpointing
* add model optimization
* add best prediction mechanism
* Application of gelu
  